# Use Mirage to Create Source Catalogs

Mirage contains some functionality for creating source catalogs that can be used when creating simulated data. This notebook shows examples of how to use these functions. When working from APT xml and pointing files, catalogs must be generated prior to calling the `yaml_generator` to create the input yaml files.

In [ ]:
import os

import numpy as np

from mirage.catalogs import catalog_generator
from mirage.catalogs import create_catalog

In [ ]:
TEST_DATA_DIR = os.path.join(os.path.dirname(__file__), 'catalogs')

## Table of Contents

stuff here

# Create Simple Catalogs from RA, Dec, magnitude lists

## Point Source Catalogs

### Create Catalog from lists of RA, Dec, and Magnitudes

In [ ]:
# Read in information from a file
example_point_source_file = os.path.join(TEST_DATA_DIRECTORY), 'catalog_input_data.txt'
table = ascii.read(example_point_source_file) ....

#### Create Lists of RA, Dec

In [ ]:
# NIRCam longwave detector width in units of degrees
nircam_lw_detector_width = 2048. * 0.062 / 3600

In [ ]:
number_of_sources = 50
ra = np.random.random(number_of_sources) * nircam_lw_detector_width + 42.2
dec = np.random.random(number_of_sources) * nircam_lw_detector_width - 24.4

#### Create Lists of Magnitudes and Associated Filters

In [ ]:
# Have target magnitudes betwee 15-25
nircam_f150w_mags = np.random.random(number_of_sources) * 10 + 15
nircam_f444w_mags = nircam_f150w_mags + np.random.random(number_of_sources) * 0.4
niriss_f090w_mags = nircam_f150w_mags + np.random.random(number_of_sources) * 0.2

#### First create the skeleton of the catalog by adding RA and Dec values

In [ ]:
ptsrc = catalog_generator.PointSourceCatalog(ra=ra, dec=dec)

#### Now add the magnitude lists one at a time

In [ ]:
ptsrc.add_magnitude_column(nircam_f150w_mags, instrument='nircam', filter_name='f150w')
ptsrc.add_magnitude_column(nircam_f444w_mags, instrument='nircam', filter_name='f444w')
ptsrc.add_magnitude_column(niriss_f090w_mags, instrument='niriss', filter_name='f090w')

#### Look at the resulting table

In [ ]:
ptsrc.table

#### Save the table as a Mirage-formatted point source catalog

In [ ]:
ptsrc.save('mirage_ptsrc_catalog_from_lists.cat')

### If position inputs are in pixel (x,y) coordinates, use the "x" and "y" keywords

In [ ]:
x = np.random.random(number_of_sources) * 2048.
y = np.random.random(number_of_sources) * 2048.

In [ ]:
ptsrc_xy = catalog_generator.PointSourceCatalog(x=x, y=y)
ptsrc_xy.add_magnitude_column(nircam_f150w_mags, instrument='nircam', filter_name='f150w')
ptsrc_xy.add_magnitude_column(nircam_f444w_mags, instrument='nircam', filter_name='f444w')
ptsrc_xy.add_magnitude_column(niriss_f090w_mags, instrument='niriss', filter_name='f090w')
ptsrc_xy.save('mirage_ptsrc_xy_catalog_from_lists.cat')

## Galaxy Catalogs

### Create input data to go into the catalog

In [ ]:
number_of_galaxies = 10
galaxy_ra = np.random.random(number_of_galaxies) * nircam_lw_detector_width + 14.2
galaxy_dec = np.random.random(number_of_galaxies) * nircam_lw_detector_width + -25.323
radius = np.random.random(number_of_galaxies) * 0.5 + 0.02
ellip = np.random.random(number_of_galaxies) * 0.8 + 0.02
posang = np.random.random(number_of_galaxies) * 359
sersic = np.random.random(number_of_galaxies) * 4 + 0.5

nircam_f356w_mags = np.random.random(number_of_galaxies) * 10 + 13
nircam_f480m_mags = nircam_f356w_mags + np.random.random(number_of_galaxies) * 0.4
niriss_f200w_mags = nircam_f356w_mags + np.random.random(number_of_galaxies) * 0.2

### Galaxy catalog object creation is similar to that for the point sources

In [ ]:
# Specify that the units for the galactic radii are arcseconds. The other option is 'pixels'
gal = catalog_generator.GalaxyCatalog(ra=galaxy_ra, dec=galaxy_dec, ellipticity=ellip, radius=radius,
                                      sersic_index=sersic, position_angle=posang, radius_units='arcsec')

In [ ]:
gal.add_magnitude_column(nircam_f356w_mags, instrument='nircam', filter_name='f356w')
gal.add_magnitude_column(nircam_f480m_mags, instrument='nircam', filter_name='f480m')
gal.add_magnitude_column(niriss_f200w_mags, instrument='niriss', filter_name='f200w')

In [ ]:
gal.table

In [ ]:
gal.save('example_galaxy_catalog.cat')

# Query Outside Sources to Create Catalogs

Define a specific pointing and sky area to use for queries

In [ ]:
central_ra = 15.56
central_dec = -34.45
catalog_width = 125  # arcseconds

## Get sources from 2MASS All-Sky Point Source Catalog in the region of interest

#### Note that in this case, 2MASS magnitudes are returned directly from the 2MASS catalog. No attempt is made to translate into magnitudes for any JWST instrument/filter

In [ ]:
twomass_cat, twomass_query = create_catalog.get_2MASS_ptsrc_catalog(central_ra, central_dec, catalog_width)

#### `get_2MASS_ptsrc_catalog` returns a tuple containing the Mirage-formatted source catalog as well as the original table produced by the query to the 2MASS database.

In [ ]:
# Mirage-formatted catalog: containing 2MASS J, H, and K magnitudes
twomass_cat.save('my_example_2mass_catalog.cat')
twomass_cat.table

In [ ]:
# Query results: containing all information on sources
twomass_query

## Get sources from WISE All-Sky Point Source Catalog in the region of interest

In [ ]:
wise_cat, wise_query = create_catalog.get_WISE_ptsrc_catalog(central_ra, central_dec, catalog_width)

In [ ]:
# Mirage-formatted catalog: containing WISE ?, ?, ? magnitudes
wise_cat.save('my_example_wise_catalog.cat')
wise_cat.table

In [ ]:
# Query results: containing all information on sources
wise_query

## Get sources from GAIA All-Sky Point Source Catalog in the region of interest

The GAIA search returns a Mirage-formatted catalog and the full results of the query from the GAIA database. It also returns tables containing 2MASS and WISE cross-matching references.

In [ ]:
gaia_cat, gaia_query, gaia_2mass_crossref, gaia_wise_crossref = \
  create_catalog.get_gaia_ptsrc_catalog(central_ra, central_dec, catalog_width)

In [ ]:
# Mirage-formatted catalog: containing GAIA ?, ?, ? magnitudes
gaia_cat.save('my_example_gaia_catalog.cat')
gaia_cat.table

In [ ]:
gaia_query

In [ ]:
gaia_2mass_crossref

In [ ]:
gaia_wise_crossref

## Create a catalog of representative stars, using the Besancon model

Full documentation on the Besancon model queried by Mirage is available here: http://model.obs-besancon.fr/

Create a dictionary containing the filters in which you would like the returned Besancon magnitudes

In [ ]:
filter_dict = {'nircam': ['F115W', 'F182M', 'F356W', 'F480M'],
               'niriss': ['F200W', 'F356W']}

You may also optionally specify seeds for the random number generators used to create the RA and Dec values for the stars. By providing seeds, the positions of the stars can be replicated in future calls

In [ ]:
ra_seed = 12345
dec_seed = 12346
seeds = (ra_seed, dec_seed)

The call to the Besancon model also requires an email address as input

In [ ]:
email = 'hilbert@stsci.edu'

K-magnitude limits for the returned catalog are also optional inputs. The defaults are to return sources with K betwen 13 and 29.

In [ ]:
limits=(13, 29)

Create the catalog, and save to a file

In [ ]:
besancon_output_file = 'example_besanson_catalog_jwst_mags.cat'
besancon_cat = create_catalog.besancon_catalog(central_ra, central_dec, catalog_width,
                                               filter_dict, coords='ra_dec',
                                               kmag_limits=limits, email=email, seeds=seeds,
                                               output_file=besancon_output_file)

As with the other catalog queries, this call returns the Mirage-formatted catalog in addition to the full query results

In [ ]:
besancon_cat.table

## Putting it all together: Search 2MASS, GAIA, WISE. Crossmatch and interpolate magnitudes to JWST filters

Create a list of filters to which the point source magnitudes will be interpolated. Currently this function supports filters from only one instrument at a time. Catalogs from multuple instruments can be combined in the next step.

In [ ]:
nrc_filter_list = ['F115W', 'F182M', 'F356W', 'F480M']

In [ ]:
nircam_cat, nircam_filters = create_catalog.get_all_catalogs(central_ra, central_dec, catalog_width,
                                                            instrument='nircam', filters=nrc_filter_list,
                                                            email=email, besancon_seeds=seeds)

In [ ]:
nis_filter_list = ['F200W', 'F356W']

In [ ]:
niriss_cat, niriss_filters = create_catalog.get_all_catalogs(central_ra, central_dec, catalog_width,
                                                             instrument='niriss', filters=nis_filter_list,
                                                             email=email, besancon_seeds=seeds)

In [ ]:
# At this point, check to see if the sources returned by Besancon are the same in the two cases.
# If so then we can combine the two catalogs. If not, we may need to think about how to handle this.

In [ ]:
# Play with Besancon transformation, since users may want a simple Besancon model and nothing else

In [ ]:
nircam_filters = create_catalog.make_filter_names('nircam', nrc_filter_list)
niriss_filters = create_catalog.make_filter_names('niriss', nis_filter_list)
all_filters = nircam_filters + niriss_filters

In [ ]:
newmags = create_catalog.transform_besancon(background, background_cat, 'nircam', all_filters)

In [ ]:
newmags.shape

In [ ]:
transformed_besancon_cat = catalog_generator.PointSourceCatalog(ra=background.table['x_or_RA'].data,
                                                                dec=background.table['y_or_Dec'].data)
for mags, filt in zip(newmags.transpose(), all_filters):
    instrument, filter_name, _ = filt.split('_')
    transformed_besancon_cat.add_magnitude_column(mags, instrument=instrument, filter_name=filter_name)

In [ ]:
transformed_besancon_cat.table

In [ ]:
transformed_besancon_cat.save('besancon_catalog.cat')

In [ ]:
filters = {'nircam': nrc_filter_list, 'niriss': nis_filter_list}
newcat = besancon_catalog(central_ra, central_dec, catalog_width, filters, coords='ra_dec',
                          kmag_limits=(13, 29), email='hilbert@stsci.edu',
                          seeds=[None, None], output_file=None)

In [ ]:
newcat.table